In [1]:
import pandas as pd
import os

import matplotlib as plt
import seaborn as sns
import numpy as np
from env import host, user, password

In [2]:
def get_connection(db, username=user, host=host, password=password):
    return f'mysql+pymysql://{username}:{password}@{host}/{db}'

In [3]:
sql = """
        SELECT prop.*,
        predictions_2017.logerror,
        predictions_2017.transactiondate,
        air.airconditioningdesc,
        arch.architecturalstyledesc,
        build.buildingclassdesc,
        heat.heatingorsystemdesc,
        land.propertylandusedesc,
        story.storydesc,
        type.typeconstructiondesc
        FROM properties_2017 prop
        JOIN (
            SELECT parcelid, MAX(transactiondate) AS max_transactiondate
            FROM predictions_2017
            GROUP BY parcelid
            ) pred USING(parcelid)
        JOIN predictions_2017 ON pred.parcelid = predictions_2017.parcelid
                          AND pred.max_transactiondate = predictions_2017.transactiondate
        LEFT JOIN airconditioningtype air USING(airconditioningtypeid)
        LEFT JOIN architecturalstyletype arch USING(architecturalstyletypeid)
        LEFT JOIN buildingclasstype build USING(buildingclasstypeid)
        LEFT JOIN heatingorsystemtype heat USING(heatingorsystemtypeid)
        LEFT JOIN propertylandusetype land USING(propertylandusetypeid)
        LEFT JOIN storytype story USING(storytypeid)
        LEFT JOIN typeconstructiontype type USING(typeconstructiontypeid)
        WHERE propertylandusedesc = "Single Family Residential"
            AND transactiondate <= '2017-12-31'
            AND prop.longitude IS NOT NULL
            AND prop.latitude IS NOT NULL
    """

df = pd.read_sql(sql, get_connection("zillow"));

##  2. Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)



In [4]:
df.columns

Index(['id', 'parcelid', 'airconditioningtypeid', 'architecturalstyletypeid',
       'basementsqft', 'bathroomcnt', 'bedroomcnt', 'buildingclasstypeid',
       'buildingqualitytypeid', 'calculatedbathnbr', 'decktypeid',
       'finishedfloor1squarefeet', 'calculatedfinishedsquarefeet',
       'finishedsquarefeet12', 'finishedsquarefeet13', 'finishedsquarefeet15',
       'finishedsquarefeet50', 'finishedsquarefeet6', 'fips', 'fireplacecnt',
       'fullbathcnt', 'garagecarcnt', 'garagetotalsqft', 'hashottuborspa',
       'heatingorsystemtypeid', 'latitude', 'longitude', 'lotsizesquarefeet',
       'poolcnt', 'poolsizesum', 'pooltypeid10', 'pooltypeid2', 'pooltypeid7',
       'propertycountylandusecode', 'propertylandusetypeid',
       'propertyzoningdesc', 'rawcensustractandblock', 'regionidcity',
       'regionidcounty', 'regionidneighborhood', 'regionidzip', 'roomcnt',
       'storytypeid', 'threequarterbathnbr', 'typeconstructiontypeid',
       'unitcnt', 'yardbuildingsqft17', 'yardb

In [5]:
df.shape

(52319, 68)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52319 entries, 0 to 52318
Data columns (total 68 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            52319 non-null  int64  
 1   parcelid                      52319 non-null  int64  
 2   airconditioningtypeid         13615 non-null  float64
 3   architecturalstyletypeid      70 non-null     float64
 4   basementsqft                  47 non-null     float64
 5   bathroomcnt                   52319 non-null  float64
 6   bedroomcnt                    52319 non-null  float64
 7   buildingclasstypeid           0 non-null      object 
 8   buildingqualitytypeid         33654 non-null  float64
 9   calculatedbathnbr             52184 non-null  float64
 10  decktypeid                    389 non-null    float64
 11  finishedfloor1squarefeet      4371 non-null   float64
 12  calculatedfinishedsquarefeet  52238 non-null  float64
 13  f

In [7]:
df.value_counts

<bound method DataFrame.value_counts of             id  parcelid  airconditioningtypeid  architecturalstyletypeid  \
0      1727539  14297519                    NaN                       NaN   
1      1387261  17052889                    NaN                       NaN   
2        11677  14186244                    NaN                       NaN   
3      2288172  12177905                    NaN                       NaN   
4       781532  12095076                    1.0                       NaN   
...        ...       ...                    ...                       ...   
52314  2274245  12412492                    NaN                       NaN   
52315   673515  11000655                    NaN                       NaN   
52316  2968375  17239384                    NaN                       NaN   
52317  1843709  12773139                    1.0                       NaN   
52318  1187175  12826780                    NaN                       NaN   

       basementsqft  bathroomcnt  b

In [8]:
df.describe()

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyyear,censustractandblock,logerror
count,5.231900e+04,5.231900e+04,13615.000000,70.00000,47.000000,52319.000000,52319.000000,33654.000000,52184.000000,389.0,...,14532.000000,81.0,5.223700e+04,5.231800e+04,52319.0,5.231800e+04,52315.000000,2068.000000,5.219800e+04,52319.000000
mean,1.496888e+06,1.299682e+07,2.439589,7.10000,678.978723,2.300015,3.300770,6.264961,2.305276,66.0,...,1.414189,1.0,1.966385e+05,5.298287e+05,2016.0,3.334946e+05,6454.802564,14.101064,6.050245e+13,0.018136
std,8.594310e+05,3.350951e+06,3.847925,2.66567,711.825226,1.022816,0.947563,1.716328,1.018076,0.0,...,0.540685,0.0,2.542880e+05,7.518358e+05,0.0,5.705157e+05,8752.550365,2.400399,1.861145e+12,0.176904
min,3.490000e+02,1.071186e+07,1.000000,2.00000,38.000000,0.000000,0.000000,1.000000,1.000000,66.0,...,1.000000,1.0,1.290000e+02,1.000000e+03,2016.0,1.610000e+02,49.180000,4.000000,6.037101e+13,-4.655420
25%,7.575810e+05,1.151017e+07,1.000000,7.00000,263.500000,2.000000,3.000000,5.000000,2.000000,66.0,...,1.000000,1.0,7.715900e+04,1.940205e+05,2016.0,7.619400e+04,2660.965000,14.000000,6.037401e+13,-0.024702
50%,1.500095e+06,1.257825e+07,1.000000,7.00000,512.000000,2.000000,3.000000,6.000000,2.000000,66.0,...,1.000000,1.0,1.319100e+05,3.740080e+05,2016.0,2.180790e+05,4650.620000,15.000000,6.037620e+13,0.006935
75%,2.241248e+06,1.413036e+07,1.000000,7.00000,809.500000,3.000000,4.000000,8.000000,3.000000,66.0,...,2.000000,1.0,2.264550e+05,6.193580e+05,2016.0,4.087795e+05,7379.320000,15.000000,6.059042e+13,0.040599
max,2.982270e+06,1.676878e+08,13.000000,21.00000,3560.000000,18.000000,14.000000,12.000000,18.000000,66.0,...,6.000000,1.0,9.164901e+06,4.906124e+07,2016.0,4.895220e+07,586639.300000,99.000000,4.830301e+14,5.262999


In [9]:
pd.set_option('display.max_columns', None)

df.head()

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertycountylandusecode,propertylandusetypeid,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,storytypeid,threequarterbathnbr,typeconstructiontypeid,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,1727539,14297519,NaN,NaN,NaN,3.5,4.0,None,NaN,3.5,NaN,NaN,3100.0,3100.0,None,None,NaN,NaN,6059.0,NaN,3.0,2.0,633.0,NaN,NaN,33634931.0,-117869207.0,4506.0,NaN,NaN,NaN,NaN,NaN,122,261.0,None,6.059063e+07,53571.0,1286.0,NaN,96978.0,0.0,NaN,1.0,NaN,NaN,NaN,NaN,1998.0,NaN,NaN,485713.0,1023282.0,2016.0,537569.0,11013.72,None,NaN,6.059063e+13,0.025595,2017-01-01,None,None,None,None,Single Family Residential,None,None
1,1387261,17052889,NaN,NaN,NaN,1.0,2.0,None,NaN,1.0,NaN,1465.0,1465.0,1465.0,None,None,1465.0,NaN,6111.0,1.0,1.0,1.0,0.0,NaN,NaN,34449266.0,-119281531.0,12647.0,NaN,NaN,NaN,NaN,NaN,1110,261.0,None,6.111001e+07,13091.0,2061.0,NaN,97099.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,1967.0,1.0,NaN,88000.0,464000.0,2016.0,376000.0,5672.48,None,NaN,6.111001e+13,0.055619,2017-01-01,None,None,None,None,Single Family Residential,None,None
2,11677,14186244,NaN,NaN,NaN,2.0,3.0,None,NaN,2.0,NaN,NaN,1243.0,1243.0,None,None,NaN,NaN,6059.0,NaN,2.0,2.0,440.0,NaN,NaN,33886168.0,-117823170.0,8432.0,1.0,NaN,NaN,NaN,1.0,122,261.0,None,6.059022e+07,21412.0,1286.0,NaN,97078.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,1962.0,1.0,NaN,85289.0,564778.0,2016.0,479489.0,6488.30,None,NaN,6.059022e+13,0.005383,2017-01-01,None,None,None,None,Single Family Residential,None,None
3,2288172,12177905,NaN,NaN,NaN,3.0,4.0,None,8.0,3.0,NaN,NaN,2376.0,2376.0,None,None,NaN,NaN,6037.0,NaN,3.0,NaN,NaN,NaN,2.0,34245180.0,-118240722.0,13038.0,1.0,NaN,NaN,NaN,1.0,0101,261.0,LCR110000*,6.037300e+07,396551.0,3101.0,NaN,96330.0,0.0,NaN,NaN,NaN,1.0,NaN,NaN,1970.0,NaN,NaN,108918.0,145143.0,2016.0,36225.0,1777.51,None,NaN,6.037300e+13,-0.103410,2017-01-01,None,None,None,Central,Single Family Residential,None,None
4,781532,12095076,1.0,NaN,NaN,3.0,4.0,None,9.0,3.0,NaN,NaN,2962.0,2962.0,None,None,NaN,NaN,6037.0,NaN,3.0,NaN,NaN,NaN,2.0,34145202.0,-118179824.0,63000.0,1.0,NaN,NaN,NaN,1.0,0101,261.0,PSR2,6.037461e+07,47019.0,3101.0,274684.0,96293.0,0.0,NaN,NaN,NaN,1.0,NaN,NaN,1950.0,NaN,NaN,276684.0,773303.0,2016.0,496619.0,9516.26,None,NaN,6.037461e+13,-0.001011,2017-01-01,Central,None,None,Central,Single Family Residential,None,None


In [10]:
df.typeconstructiondesc.value_counts

<bound method IndexOpsMixin.value_counts of 0        None
1        None
2        None
3        None
4        None
         ... 
52314    None
52315    None
52316    None
52317    None
52318    None
Name: typeconstructiondesc, Length: 52319, dtype: object>

  ## Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.


In [13]:
df.isnull().head()

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertycountylandusecode,propertylandusetypeid,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,storytypeid,threequarterbathnbr,typeconstructiontypeid,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,False,False,True,True,True,False,False,True,True,False,True,True,False,False,True,True,True,True,False,True,False,False,False,True,True,False,False,False,True,True,True,True,True,False,False,True,False,False,False,True,False,False,True,False,True,True,True,True,False,True,True,False,False,False,False,False,True,True,False,False,False,True,True,True,True,False,True,True
1,False,False,True,True,True,False,False,True,True,False,True,False,False,False,True,True,False,True,False,False,False,False,False,True,True,False,False,False,True,True,True,True,True,False,False,True,False,False,False,True,False,False,True,True,True,True,True,True,False,False,True,False,False,False,False,False,True,True,False,False,False,True,True,True,True,False,True,True
2,False,False,True,True,True,False,False,True,True,False,True,True,False,False,True,True,True,True,False,True,False,False,False,True,True,False,False,False,False,True,True,True,False,False,False,True,False,False,False,True,False,False,True,True,True,True,True,True,False,False,True,False,False,False,False,False,True,True,False,False,False,True,True,True,True,False,True,True
3,False,False,True,True,True,False,False,True,False,False,True,True,False,False,True,True,True,True,False,True,False,True,True,True,False,False,False,False,False,True,True,True,False,False,False,False,False,False,False,True,False,False,True,True,True,False,True,True,False,True,True,False,False,False,False,False,True,True,False,False,False,True,True,True,False,False,True,True
4,False,False,False,True,True,False,False,True,False,False,True,True,False,False,True,True,True,True,False,True,False,True,True,True,False,False,False,False,False,True,True,True,False,False,False,False,False,False,False,False,False,False,True,True,True,False,True,True,False,True,True,False,False,False,False,False,True,True,False,False,False,False,True,True,False,False,True,True


In [15]:
df.isnull().sum()[:20]

id                                  0
parcelid                            0
airconditioningtypeid           38704
architecturalstyletypeid        52249
basementsqft                    52272
bathroomcnt                         0
bedroomcnt                          0
buildingclasstypeid             52319
buildingqualitytypeid           18665
calculatedbathnbr                 135
decktypeid                      51930
finishedfloor1squarefeet        47948
calculatedfinishedsquarefeet       81
finishedsquarefeet12              246
finishedsquarefeet13            52319
finishedsquarefeet15            52319
finishedsquarefeet50            47948
finishedsquarefeet6             52154
fips                                0
fireplacecnt                    45086
dtype: int64

In [16]:
df.shape[0]

52319

In [11]:
nulls_col = pd.DataFrame({'num_rows_missing': df.isnull().sum(), 
              'percent_rows_missing': (df.isnull().sum() / df.shape[0] * 100)})

nulls_col.sort_values(by='num_rows_missing', ascending=False).head(10)

,num_rows_missing,percent_rows_missing
buildingclassdesc,52319,100.000000
buildingclasstypeid,52319,100.000000
finishedsquarefeet15,52319,100.000000
finishedsquarefeet13,52319,100.000000
storytypeid,52272,99.910166
basementsqft,52272,99.910166
storydesc,52272,99.910166
yardbuildingsqft26,52256,99.879585
architecturalstyletypeid,52249,99.866205
architecturalstyledesc,52249,99.866205


In [20]:


df.isnull().sum()

id                              0
parcelid                        0
airconditioningtypeid       38704
architecturalstyletypeid    52249
basementsqft                52272
                            ...  
buildingclassdesc           52319
heatingorsystemdesc         18470
propertylandusedesc             0
storydesc                   52272
typeconstructiondesc        52243
Length: 68, dtype: int64

In [24]:
(df.isnull().sum()/df.shape[0]*100)[:30]

id                                0.000000
parcelid                          0.000000
airconditioningtypeid            73.976949
architecturalstyletypeid         99.866205
basementsqft                     99.910166
bathroomcnt                       0.000000
bedroomcnt                        0.000000
buildingclasstypeid             100.000000
buildingqualitytypeid            35.675376
calculatedbathnbr                 0.258032
decktypeid                       99.256484
finishedfloor1squarefeet         91.645483
calculatedfinishedsquarefeet      0.154819
finishedsquarefeet12              0.470192
finishedsquarefeet13            100.000000
finishedsquarefeet15            100.000000
finishedsquarefeet50             91.645483
finishedsquarefeet6              99.684627
fips                              0.000000
fireplacecnt                     86.175194
fullbathcnt                       0.258032
garagecarcnt                     65.626254
garagetotalsqft                  65.626254
hashottubor

In [ ]:
#prepare 

 ## Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer.

In [25]:
df.propertylandusedesc.info()

<class 'pandas.core.series.Series'>
RangeIndex: 52319 entries, 0 to 52318
Series name: propertylandusedesc
Non-Null Count  Dtype 
--------------  ----- 
52319 non-null  object
dtypes: object(1)
memory usage: 408.9+ KB


In [26]:
df.propertylandusedesc.head()

0    Single Family Residential
1    Single Family Residential
2    Single Family Residential
3    Single Family Residential
4    Single Family Residential
Name: propertylandusedesc, dtype: object

In [27]:
df.propertylandusedesc.unique()

array(['Single Family Residential'], dtype=object)

## Create a function that will drop rows or columns based on the percent of values that are missing: handle_missing_values(df, prop_required_column, prop_required_row).

The input:
- A dataframe
- A number between 0 and 1 that represents the proportion, for each column, of rows with non-missing values required to keep the column. i.e. if prop_required_column = .6, then you are requiring a column to have at least 60% of values not-NA (no more than 40% missing).
- A number between 0 and 1 that represents the proportion, for each row, of columns/variables with non-missing values required to keep the row. For example, if prop_required_row = .75, then you are requiring a row to have at least 75% of variables with a non-missing value (no more that 25% missing).

The output:
- The dataframe with the columns and rows dropped as indicated. Be sure to drop the columns prior to the rows in your function.

hint:
- Look up the dropna documentation.
- You will want to compute a threshold from your input values (prop_required) and total number of rows or columns.

In [28]:
def nulls_by_columns(df):
    return pd.concat([
        df.isna().sum().rename('count'),
        df.isna().mean().rename('percent')
    ], axis=1)

In [ ]:
cols_to_remove = ['heatingorsystemtypeid','parcelid','storytypeid','typeconstructiontypeid','airconditioningtypeid','propertylandusetypeid','architecturalstyletypeid','id','buildingclasstypeid','buildingqualitytypeid','decktypeid','pooltypeid10','pooltypeid2','pooltypeid7','taxamount','taxdelinquencyflag','taxdelinquencyyear','id']


def remove_columns(df, cols_to_remove):
    df = df.drop(columns=cols_to_remove)
    return df

In [ ]:
remove_columns(df, cols_to_remove)

In [29]:
def handle_missing_values(df, prop_required_columns=0.6, prop_required_row=0.75):
    threshold = int(round(prop_required_columns * len(df.index), 0))
    df = df.dropna(axis=1, thresh=threshold) #1, or ‘columns’ : Drop columns which contain missing value
    threshold = int(round(prop_required_row * len(df.columns), 0))
    df = df.dropna(axis=0, thresh=threshold) #0, or ‘index’ : Drop rows which contain missing values.
    return df

In [34]:
handle_missing_values(df, prop_required_columns=0.60, prop_required_row=0.75)

,id,parcelid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fullbathcnt,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,propertycountylandusecode,propertylandusetypeid,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidzip,roomcnt,unitcnt,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,logerror,transactiondate,heatingorsystemdesc,propertylandusedesc
0,1727539,14297519,3.5,4.0,NaN,3.5,3100.0,3100.0,6059.0,3.0,NaN,33634931.0,-117869207.0,4506.0,122,261.0,None,6.059063e+07,53571.0,1286.0,96978.0,0.0,NaN,1998.0,485713.0,1023282.0,2016.0,537569.0,11013.72,6.059063e+13,0.025595,2017-01-01,None,Single Family Residential
1,1387261,17052889,1.0,2.0,NaN,1.0,1465.0,1465.0,6111.0,1.0,NaN,34449266.0,-119281531.0,12647.0,1110,261.0,None,6.111001e+07,13091.0,2061.0,97099.0,5.0,NaN,1967.0,88000.0,464000.0,2016.0,376000.0,5672.48,6.111001e+13,0.055619,2017-01-01,None,Single Family Residential
2,11677,14186244,2.0,3.0,NaN,2.0,1243.0,1243.0,6059.0,2.0,NaN,33886168.0,-117823170.0,8432.0,122,261.0,None,6.059022e+07,21412.0,1286.0,97078.0,6.0,NaN,1962.0,85289.0,564778.0,2016.0,479489.0,6488.30,6.059022e+13,0.005383,2017-01-01,None,Single Family Residential
3,2288172,12177905,3.0,4.0,8.0,3.0,2376.0,2376.0,6037.0,3.0,2.0,34245180.0,-118240722.0,13038.0,0101,261.0,LCR110000*,6.037300e+07,396551.0,3101.0,96330.0,0.0,1.0,1970.0,108918.0,145143.0,2016.0,36225.0,1777.51,6.037300e+13,-0.103410,2017-01-01,Central,Single Family Residential
4,781532,12095076,3.0,4.0,9.0,3.0,2962.0,2962.0,6037.0,3.0,2.0,34145202.0,-118179824.0,63000.0,0101,261.0,PSR2,6.037461e+07,47019.0,3101.0,96293.0,0.0,1.0,1950.0,276684.0,773303.0,2016.0,496619.0,9516.26,6.037461e+13,-0.001011,2017-01-01,Central,Single Family Residential
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52314,2274245,12412492,2.0,4.0,6.0,2.0,1633.0,1633.0,6037.0,2.0,2.0,33870815.0,-118070858.0,4630.0,0100,261.0,ATR16000*,6.037555e+07,30267.0,3101.0,96204.0,0.0,1.0,1962.0,125466.0,346534.0,2016.0,221068.0,4175.08,6.037555e+13,0.001082,2017-09-19,Central,Single Family Residential
52315,673515,11000655,2.0,2.0,6.0,2.0,1286.0,1286.0,6037.0,2.0,2.0,34245368.0,-118282383.0,47405.0,0100,261.0,LARE40,6.037101e+07,12447.0,3101.0,96284.0,0.0,1.0,1940.0,70917.0,354621.0,2016.0,283704.0,4478.43,6.037101e+13,0.020615,2017-09-20,Central,Single Family Residential
52316,2968375,17239384,2.0,4.0,NaN,2.0,1612.0,1612.0,6111.0,2.0,NaN,34300140.0,-118706327.0,12105.0,1111,261.0,None,6.111008e+07,27110.0,2061.0,97116.0,7.0,NaN,1964.0,50683.0,67205.0,2016.0,16522.0,1107.48,6.111008e+13,0.013209,2017-09-21,None,Single Family Residential
52317,1843709,12773139,1.0,3.0,4.0,1.0,1032.0,1032.0,6037.0,1.0,2.0,34040895.0,-118038169.0,5074.0,0100,261.0,SER1*,6.037434e+07,36502.0,3101.0,96480.0,0.0,1.0,1954.0,32797.0,49546.0,2016.0,16749.0,876.43,6.037434e+13,0.037129,2017-09-21,Central,Single Family Residential


In [35]:
df.unitcnt.value_counts

<bound method IndexOpsMixin.value_counts of 0        NaN
1        NaN
2        NaN
3        1.0
4        1.0
        ... 
52314    1.0
52315    1.0
52316    NaN
52317    1.0
52318    1.0
Name: unitcnt, Length: 52319, dtype: float64>

In [ ]:
nulls_col = pd.DataFrame({'num_rows_missing': df.isnull().sum(), 
              'percent_rows_missing': (df.isnull().sum() / df.shape[0] * 100)})

nulls_col.sort_values(by='num_rows_missing', ascending=False).head(10)

In [ ]:
#Define function to drop columns/rows based on proportion of nulls
def null_dropper(df, prop_required_column, prop_required_row):
    
    prop_null_column = 1 - prop_required_column
    
    for col in list(df.columns):
        
        null_sum = df[col].isna().sum()
        null_pct = null_sum / df.shape[0]
        
        if null_pct > prop_null_column:
            df.drop(columns=col, inplace=True)
            
    row_threshold = int(prop_required_row * df.shape[1])
    
    df.dropna(axis=0, thresh=row_threshold, inplace=True)
    
    return df

In [ ]:
clean_df = null_dropper(df, 0.75, 0.75)

clean_df.head()

In [ ]:
#Check shape of resulting dataframe
clean_df.shape

In [ ]:
#Save a copy of the clean csv to my local machine
clean_df.to_csv('zillow.csv')

## Mall Customers

In [ ]:
sql = """SELECT * FROM customers;"""

df2 = pd.read_sql(sql, get_connection("mall_customers"));

In [ ]:
df2.head()

Summarize data (include distributions and descriptive statistics).

In [ ]:
df2.describe()

Detect outliers using IQR.

In [ ]:
def get_upper_outliers(s, k=1.5):
    '''
    Given a series and a cutoff value, k, returns the upper outliers for the
    series.

    The values returned will be either 0 (if the point is not an outlier), or a
    number that indicates how far away from the upper bound the observation is.
    '''
    q1, q3 = s.quantile([.25, 0.75])
    iqr = q3 - q1
    upper_bound = q3 + k * iqr
    return s.apply(lambda x: max([x - upper_bound, 0]))

In [ ]:
def add_upper_outlier_columns(df, k=1.5):
    '''
    Add a column with the suffix _outliers for all the numeric columns
    in the given dataframe.
    '''
    for col in df.select_dtypes('number'):
        df[col + '_outliers_upper'] = get_upper_outliers(df[col], k)
    return df

In [ ]:
df3 = add_upper_outlier_columns(df2)


In [ ]:
df3.head()


In [ ]:
outlier_cols = [col for col in df3.columns if col.endswith('_outliers_upper')]
for col in outlier_cols:
    print(col, ': ')
    subset = df[col][df[col] > 0]
    print(f'Number of Observations Above Upper Bound: {subset.count()}', '\n')
    print(subset.describe())
    print('------', '\n')

Split data into train, validate, and test.

In [ ]:
from sklearn.model_selection import train_test_split

def split_data(df):
    '''
    take in a DataFrame and return train, validate, and test DataFrames; stratify on taxvaluedollarcnt.
    return train, validate, test DataFrames.
    '''


    train_validate, test = train_test_split(df, test_size=.2, random_state=123)
    train, validate = train_test_split(train_validate, 
                                       test_size=.3, 
                                       random_state=123)
    return train, validate, test

train, validate, test = split_data(df3)

In [ ]:
train.shape, validate.shape, test.shape

Encode categorical columns using a one hot encoder (pd.get_dummies).

In [ ]:
dummy_df = pd.get_dummies(train['gender'],dummy_na=False, drop_first = True)
#append to add encoded col to df
train = pd.concat([train, dummy_df], axis=1)

In [ ]:
train.head()

Handles missing values.

In [ ]:
train.isnull().sum()

In [ ]:
cols_to_remove = ['customer_id_outliers_upper','age_outliers_upper','annual_income_outliers_upper','spending_score_outliers_upper','customer_id_outliers_upper_outliers_upper', 'age_outliers_upper_outliers_upper','annual_income_outliers_upper_outliers_upper', 'spending_score_outliers_upper_outliers_upper']
train.drop(columns=cols_to_remove)

Scaling

In [ ]:
import sklearn.preprocessing
from scipy import stats
import matplotlib.pyplot as plt


x = stats.skewnorm(7).rvs(1500) * 10 + 100
x = x.reshape(-1, 1)
#I first again split, to have a set of data that is not scaled and one
#that is scaled:
x_train_and_validate, x_test = train_test_split(x, random_state=123)
x_train, x_validate = train_test_split(x_train_and_validate)

scaler = sklearn.preprocessing.MinMaxScaler()
# Note that we only call .fit with the training data,
# but we use .transform to apply the scaling to all the data splits.
scaler.fit(x_train)

x_train_scaled = scaler.transform(x_train)
x_validate_scaled = scaler.transform(x_validate)
x_test_scaled = scaler.transform(x_test)

plt.figure(figsize=(13, 6))
plt.subplot(121)
plt.hist(x_train, bins=25, ec='black')
plt.title('Original')
plt.subplot(122)
plt.hist(x_train_scaled, bins=25, ec='black')
plt.title('Scaled')